In [14]:
!pip install catppuccin-jupyterlab

  Obtaining dependency information for catppuccin-jupyterlab from https://files.pythonhosted.org/packages/a5/00/090fa6be87d7caacc1011e514c38389ef146adf76b7b39556f069c028cd9/catppuccin_jupyterlab-0.2.0-py3-none-any.whl.metadata


In [1]:
import polars as pl, polars.selectors as cs, numpy as np, seaborn as sns
import matplotlib.pyplot as plt, matplotlib.colors as clr
from tqdm import tqdm
import Valorant.functions as val

In [2]:
df = pl.read_csv('brutos\\todos_os_dados_final.csv', infer_schema_length=0)
df = df.select(
    ~cs.contains(('Map '))).drop(
        ['', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 1'])
df = val.processador(df)

100%|███████████████████████████████████████████████████████████████████████████| 20600/20600 [00:45<00:00, 451.03it/s]


In [3]:
media_jogadores = False
media_jogos = False

part_consid = 6
n = part_consid*5

Teams = df.select('Team', 'Player-name')
Teams = Teams.group_by('Team').count().filter(
    pl.col('count') >= n).get_column('Team').to_list()
Teams.remove(None)
Teams.remove('VKS')

In [4]:
jogos_por_time, df_por_time=  val.indices_df(df, Teams, part_consid, media_jogadores, primeiro=True)

for time in Teams:
    assert df_por_time[time].shape[0] > 0, ('deu errado no time: ', time)

treino, objetivo = val.gerar_treino(df, Teams, part_consid, jogos_por_time, df_por_time, media_jogadores, media_jogos)

# treino_media = val.media_jogadores_final(treino)

100%|████████████████████████████████████████████████████████████████████████████| 15068/15068 [14:33<00:00, 17.25it/s]


In [5]:
treino

R_0_0,R_0_1,R_0_2,R_0_3,R_0_4,R_0_5,R_1_0,R_1_1,R_1_2,R_1_3,R_1_4,R_1_5,R_2_0,R_2_1,R_2_2,R_2_3,R_2_4,R_2_5,R_3_0,R_3_1,R_3_2,R_3_3,R_3_4,R_3_5,R_4_0,R_4_1,R_4_2,R_4_3,R_4_4,R_4_5,ACS_0_0,ACS_0_1,ACS_0_2,ACS_0_3,ACS_0_4,ACS_0_5,ACS_1_0,…,FD_4_1_0,FD_4_1_1,FD_4_1_2,FD_4_1_3,FD_4_1_4,FD_4_1_5,FK-FD_0_1_0,FK-FD_0_1_1,FK-FD_0_1_2,FK-FD_0_1_3,FK-FD_0_1_4,FK-FD_0_1_5,FK-FD_1_1_0,FK-FD_1_1_1,FK-FD_1_1_2,FK-FD_1_1_3,FK-FD_1_1_4,FK-FD_1_1_5,FK-FD_2_1_0,FK-FD_2_1_1,FK-FD_2_1_2,FK-FD_2_1_3,FK-FD_2_1_4,FK-FD_2_1_5,FK-FD_3_1_0,FK-FD_3_1_1,FK-FD_3_1_2,FK-FD_3_1_3,FK-FD_3_1_4,FK-FD_3_1_5,FK-FD_4_1_0,FK-FD_4_1_1,FK-FD_4_1_2,FK-FD_4_1_3,FK-FD_4_1_4,FK-FD_4_1_5,Game differential
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
1.4,0.65,1.08,1.63,1.22,0.62,1.42,1.03,1.24,0.8,1.1,1.44,1.97,0.7,1.22,1.45,1.13,1.14,1.69,1.0,1.14,1.35,1.11,1.35,1.25,0.91,0.61,1.37,0.98,1.08,214.0,137.0,174.0,341.0,222.0,130.0,230.0,…,2.0,4.0,1.0,5.0,6.0,3.0,1.0,-4.0,-1.0,0.0,-1.0,1.0,-1.0,0.0,-1.0,5.0,-6.0,3.0,3.0,-4.0,3.0,0.0,4.0,2.0,-2.0,-1.0,1.0,4.0,1.0,5.0,4.0,-3.0,1.0,1.0,1.0,7.0,0
1.79,1.32,1.4,1.27,0.99,0.94,1.43,1.66,1.49,1.29,1.15,1.29,1.08,1.33,1.21,1.3,1.32,1.31,1.44,1.6,1.69,1.25,1.1,0.87,1.0,1.11,1.15,0.89,1.18,0.82,431.0,235.0,321.0,286.0,214.0,251.0,241.0,…,0.0,4.0,2.0,4.0,9.0,0.0,0.0,3.0,-4.0,-5.0,6.0,-1.0,4.0,7.0,6.0,8.0,6.0,4.0,3.0,0.0,-2.0,-2.0,2.0,3.0,1.0,1.0,1.0,-1.0,0.0,3.0,3.0,-3.0,1.0,-2.0,-5.0,5.0,0
1.53,1.05,0.89,1.07,0.71,0.8,1.22,1.31,1.3,1.41,0.7,1.04,1.48,1.21,0.91,1.35,0.9,0.98,1.45,1.03,1.08,0.98,0.84,0.94,1.54,1.1,1.17,1.34,0.94,1.09,276.0,205.0,173.0,212.0,141.0,169.0,164.0,…,7.0,5.0,12.0,3.0,7.0,4.0,-3.0,4.0,-2.0,1.0,3.0,-5.0,5.0,2.0,0.0,-6.0,3.0,-1.0,1.0,3.0,2.0,-1.0,1.0,1.0,5.0,1.0,7.0,-7.0,-1.0,4.0,0.0,-2.0,-8.0,4.0,4.0,0.0,0
1.07,0.66,0.98,0.92,1.0,0.93,1.65,1.57,1.41,1.2,1.33,0.92,1.29,1.05,1.03,1.04,1.05,0.68,1.19,1.43,1.15,1.2,1.09,0.8,1.11,1.17,1.28,0.68,1.3,0.81,195.0,127.0,198.0,185.0,194.0,191.0,334.0,…,4.0,7.0,1.0,6.0,4.0,6.0,2.0,0.0,0.0,-5.0,0.0,-3.0,4.0,0.0,-2.0,-2.0,3.0,-1.0,2.0,-1.0,1.0,-1.0,3.0,6.0,1.0,1.0,0.0,-2.0,1.0,5.0,1.0,2.0,4.0,-4.0,2.0,1.0,0
1.32,1.4,1.27,0.99,0.94,1.43,1.66,1.49,1.29,1.15,1.29,1.24,1.33,1.21,1.3,1.32,1.31,1.08,1.6,1.69,1.25,1.1,0.87,1.06,1.11,1.15,0.89,1.18,0.82,0.93,235.0,321.0,286.0,214.0,251.0,364.0,305.0,…,7.0,1.0,6.0,4.0,6.0,8.0,0.0,0.0,-5.0,0.0,-3.0,0.0,0.0,-2.0,-2.0,3.0,-1.0,-1.0,-1.0,1.0,-1.0,3.0,6.0,-9.0,1.0,0.0,-2.0,1.0,5.0,0.0,2.0,4.0,-4.0,2.0,1.0,-2.0,-1
1.6,0.87,1.46,1.07,1.33,1.07,1.69,0.94,1.18,0.96,1.52,1.27,1.16,0.71,1.58,1.04,1.21,1.32,1.47,0.88,1.87,1.44,1.56,1.62,1.25,0.79,1.45,1.24,1.24,1.04,245.0,200.0,263.0,192.0,243.0,185.0,263.0,…,5.0,6.0,3.0,3.0,1.0,4.0,0.0,-1.0,1.0,-1.0,1.0,0.0,2.0,-4.0,-5.0,1.0,5.0,2.0,-3.0,2.0,-2.0,-1.0,-4.0,0.0,1.0,0.0,-4.0,3.0,0.0,-5.0,3.0,1.0,-2.0,2.0,4.0,12.0,0
0.93,1.01,1.12,1.02,1.21,1.29,0.91,1.19,0.95,0.8,1.19,1.33,1.44,1.21,1.21,1.23,1.45,1.45,0.83,0.71,0.96,0.96,1.04,1.34,1.55,1.29,1.2,1.0,1.58,1.0,127.0,192.0,206.0,240.0,238.0,206.0,270.0,…,6.0,5.0,0.0,2.0,3.0,2.0,2.0,3.0,2.0,3.0,1.0,1.0,0.0,4.0,9.0,7.0,6.0,8.0,-1.0,-3.0,2.0,0.0,5.0,0.0,0.0,3.0,-1.0,0.0,3.0,-3.0,-4.0,-1.0,4.0,2.0,1.0,4.0,0
1.46,0.69,0.85,0.82,1.39,1.37,0.7,0.79,0.9,0.82,1.06,1.29,1.12,0.98,1.07,1.18,1.28,1.97,1.25,0.93,0.97,1.02,1.63,1.58,1.22,1.17,1.09,1.09,1.34,0.93,244.0,132.0,153.0,182.0,266.0,308.0,110.0,…,7.0,1.0,4.0,3.0,3.0,1.0,-2.0,6.0,0.0,5.0,3.0,7.0,1.0,0.0,1.0,3.0,-1.0,5.0,-2.0,2.0,-1.0,-1.0,2.0,1.0,4.0,3.0,4.0,6.0,3.0,-1.0,-2.0,1.0,-1.0,0.0,-1.0,4.0,-1
0.7,1.19,1.4,0.98,1.17,0.82,0.71,0.83,1.68,0.98,1.09,1.04,0.69,1.13,1.43,1.28,1.23,0.98,0.55,1.2,1.35,1.02,1.28,0.85,0.75,1.32,1.3,1.07,1.59,1.25,174.0,214.0,182.0,204.0,183.0,168.0,154.0,…,1.0,1.0,5.0,7.0,1.0,2.0,-2.0,2.0,5.0,2.0,8.0,2.0,-4.0,0.0,1.0,3.0,2.0,4.0

In [6]:
df2 = pl.read_csv('brutos/segunda_leva.csv', infer_schema_length=0)
df2 = df2.select(
    ~cs.contains(('Map '))).drop(['', 'Unnamed: 1'])

df2 = val.processador(df2)

jogos_por_time2 = val.indices_df(df2, Teams, part_consid, media_jogadores, primeiro=False)

valid1, objetivo_valid1 = val.gerar_treino(df2, Teams, part_consid, jogos_por_time2, df_por_time, media_jogadores, media_jogos)

# valid1 = val.media_jogadores_final(valid1)

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:15<00:00, 15.67it/s]


In [7]:
df3 = pl.read_csv('brutos/terceira_leva.csv', infer_schema_length=0)
df3 = df3.select(
    ~cs.contains(('Map '))).drop(['', 'Unnamed: 1'])

df3 = val.processador(df3)

jogos_por_time3 = val.indices_df(df3, Teams, part_consid, media_jogadores, primeiro=False)

valid2, objetivo_valid2 = val.gerar_treino(df3, Teams, part_consid, jogos_por_time3, df_por_time, media_jogadores, media_jogos)

# valid2 = val.media_jogadores_final(valid2)

100%|████████████████████████████████████████████████████████████████████████████████| 275/275 [00:14<00:00, 19.34it/s]


In [8]:
validacao = pl.concat([valid1, valid2])
objetivo_validacao = pl.concat([objetivo_valid1, objetivo_valid2])

In [10]:
pl.concat([treino, objetivo], how='horizontal').write_csv('processados/treino-teste_full_'+str(part_consid)+'_partidas.csv')
pl.concat([validacao, objetivo_validacao], how='horizontal').write_csv('processados/valida_full_'+str(part_consid)+'_partidas.csv')

In [ ]:
# plt.figure(figsize=(15,15)) 
# sns.heatmap(pl.concat([treino_media, objetivo], how='horizontal').corr(), cmap='cividis', xticklabels=treino_media.columns+['resultado'], yticklabels=treino_media.columns+['resultado'])

In [ ]:
from numba import jit


@jit(nopython=True)
def correlation_ratio(categories, values):
    ssw = 0
    ssb = 0
    for category in set(categories):
        subgroup = values[np.where(categories == category)[0]]
        ssw += sum((subgroup-np.mean(subgroup))**2)
        ssb += len(subgroup)*(np.mean(subgroup)-np.mean(values))**2

    return (ssb / (ssb + ssw))**.5

In [ ]:
import random
i = 0
while i < 0.5:
    col1, col2 = random.sample(treino_media.columns, 2)
    results = np.empty([1000, 100])
    for alpha in range(1, 10000, 10):
        for beta in range(1, 1000, 10):
            soma = np.array((treino_media
                             .with_columns(pl.sum(alpha*pl.col(col1)/10, beta*pl.col(col2)/10))
                             )['sum'])
            results[alpha//10][beta //
                               10] = correlation_ratio(np.ravel(np.array(objetivo)), soma)
    i = results.max()
    print(i, col1, col2)

In [ ]:
X = np.arange(1, 10000, 10)
Y = np.arange(1, 1000, 10)
X, Y = np.meshgrid(X, Y)

fig, (ax1, ax2) = plt.subplots(2, 2, figsize=(
    15, 15), subplot_kw={"projection": "3d"})
ax1[0].plot_surface(X, Y, results.T, vmin=results.min(), cmap='cividis')
ax1[0].view_init(30, 190)
ax2[0].plot_surface(X, Y, results.T, vmin=results.min(), cmap='cividis')
ax2[0].view_init(30, 280)
ax1[1].plot_surface(X, Y, results.T, vmin=results.min(), cmap='cividis')
ax1[1].view_init(30, 100)
ax2[1].plot_surface(X, Y, results.T, vmin=results.min(), cmap='cividis')
ax2[1].view_init(30, 10)
[ax.set_xlabel('alpha') for ax in list(ax1) + list(ax2)]
[ax.set_ylabel('beta') for ax in list(ax1) + list(ax2)]


plt.show()

In [3]:
(pl.read_csv('jogos_por_dia\\2023-08-16.csv', infer_schema_length=0).select(
    #         ~cs.contains(('Map', '-t','-ct'))
    ~cs.contains(('Map '))
)
    .drop(['', 'Unnamed: 1'])
    .with_columns(pl.col('D').str.extract(r'(\d+)', 0))
    .with_columns(pl.col(['R', 'ACS', 'K', 'A', 'ADR', 'FK', 'FD', 'HS%', 'KAST', '+/–', '+/–.1']).str.split('  ').list.first())
    .with_columns(pl.col('HS%', 'KAST').str.strip_chars('%'))
    .with_columns(pl.col(['R', 'ACS', 'D', 'K', 'A', 'ADR', 'FK', 'FD', 'HS%', 'KAST', '+/–', '+/–.1']).cast(pl.Float64))
    .with_columns(pl.col('HS%', 'KAST')/100)
    .with_columns(pl.col('Unnamed: 0').str.split(' ').alias('fields'))
    .with_columns(pl.col('fields').list.reverse())
    .with_columns(pl.col('fields').list.first().alias('Team'))
    .with_columns(pl.col('fields').list.slice(1).list.join(' ').alias('Player-name'))
    .drop(['fields', 'Unnamed: 0'])
    .rename({'+/–': 'K-D', '+/–.1': 'FK-FD'})
)

R,ACS,K,D,A,K-D,KAST,ADR,HS%,FK,FD,FK-FD,scores_team_1,scores_team_2,Team,Player-name
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str
1.06,222.0,33.0,28.0,7.0,5.0,0.74,159.0,0.23,3.0,2.0,1.0,"""0.0""","""2.0""","""FNC""","""Alfajer"""
1.0,211.0,29.0,29.0,6.0,0.0,0.56,142.0,0.15,14.0,10.0,4.0,"""11.0""","""13.0""","""FNC""","""Derke"""
0.99,171.0,29.0,28.0,10.0,1.0,0.74,117.0,0.2,0.0,2.0,-2.0,"""6.0""","""13.0""","""FNC""","""Leo"""
0.9,162.0,21.0,30.0,18.0,-9.0,0.58,104.0,0.16,4.0,5.0,-1.0,null,null,"""FNC""","""Boaster"""
0.67,141.0,19.0,34.0,19.0,-15.0,0.63,99.0,0.18,1.0,2.0,-1.0,null,null,"""FNC""","""Chronicle"""
1.33,234.0,38.0,25.0,9.0,13.0,0.79,147.0,0.27,6.0,2.0,4.0,null,null,"""LOUD""","""aspas"""
1.23,223.0,34.0,25.0,5.0,9.0,0.7,146.0,0.31,5.0,6.0,-1.0,null,null,"""LOUD""","""Less"""
1.17,220.0,35.0,25.0,13.0,10.0,0.79,140.0,0.26,5.0,1.0,4.0,null,null,"""LOUD""","""cauanzin"""
0.96,136.0,20.0,24.0,26.0,-4.0,0.74,94.0,0.31,2.0,5.0,-3.0,null,null,"""LOUD""","""tuyz"""
